# Translator Use Case Question 4: Multiple situations

2020-11-26 update: Hint module results changed so the pick selection is changed. 

## Understanding the Question

The Translator Use Case Question #4 is:    

> For a patient with disease X, what are some factors (such as genetic features, comorbidities, etc) that could cause sensitivity or resistance to drug Y? 

We interpret the Translator Use Case Question to be about several somewhat-related issues:
* in the context of cancer, drug sensitivity means that the drug has the desired effect of killing or inhibiting cancerous cells and drug resistance means that the drug does not have this desired effect (this resistance can be pre-existing or can develop with time / exposure to the drug). [Reference](https://www.merckmanuals.com/home/drugs/factors-affecting-response-to-drugs/tolerance-and-resistance-to-drugs).
    * There are [multiple overarching factors](https://www.nature.com/articles/s41586-019-1730-1) to drug resistance in this context.
* in the context of pathogens and the diseases they cause, drug sensitivity (similar to the cancer context) means that the drug has the desired effect of killing or inhibiting the pathogens and drug resistance (similar to the cancer context) means that the drug does not have this desired effect. [Reference](https://www.merckmanuals.com/home/drugs/factors-affecting-response-to-drugs/tolerance-and-resistance-to-drugs). 
* outside of the above contexts, drug sensitivity often means that [the patient is unable to tolerate the drug due to adverse events](https://en.wikipedia.org/wiki/Drug_intolerance). These adverse events could be [drug allergies / hypersensitivity reactions, drug side effects, drug intolerances](https://www.racgp.org.au/afp/2013/januaryfebruary/adverse-drug-reactions/), or [adverse drug-disease interactions](https://www.drugs.com/drug_interactions.html). Drug tolerance can refer to [patients who require greater doses of a drug](https://www.merckmanuals.com/professional/clinical-pharmacology/factors-affecting-response-to-drugs/pharmacogenetics) to achieve therapeutic concentrations and effects. 
    * Genetic factors, environmental factors, and developmental factors affect drug response and metabolism. [Reference](https://www.merckmanuals.com/professional/clinical-pharmacology/factors-affecting-response-to-drugs/pharmacogenetics). 
    * This question then begins to overlap with Translator Use Case Question #5. 

**In this notebook, we will tackle situations (1) and (3) above, exploring genes involved with both the disease and the drug. 

**We therefore decided to reframe this question and ask:**
> What genes are associated with both `Disease` X AND `ChemicalSubstance` Y?

BioThings Explorer (BTE) can answer two classes of queries -- "EXPLAIN" and "PREDICT". This Question fits the EXPLAIN  template of starting with **a specific biomedical entity** (a specific `Disease` X) and finding indirect relationships with **another specific biomedical entity** (a specific `ChemicalSubstance` Y).

## Step 0: Load BTE modules, notebook functions

In [1]:
## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# import modules from biothings_explorer
from biothings_explorer.hint import Hint
from biothings_explorer.user_query_dispatcher import FindConnection

## show time that this notebook was executed 
from datetime import datetime

## packages to work with objects 
import re

## to get around jupyter notebook bug
import nest_asyncio
nest_asyncio.apply() 

In [2]:
## functions to add to modules?
def hint_display(query, hint_result):
    """
    show the type, name, number of IDs for all results returned by the query
    
    :param: query: string used in hint query
    :param: hint_result: object returned from hint query, a dictionary of lists of dictionaries
    
    Returns: None
    """
    ## function needs to be rewritten if it's going to give the exact index of each object within its type 
    display = ['type', 'name']  ## replace with the parts of the BioThings object you want to see
    concise_results = []
    for BT_type, result in hint_result.items():
        if result:  ## basically if it's not empty
            for items in result:
                ## number of identifiers per object: number of keys - 4 (name, primary, display, type)
                temp = len(items) - 4
                concise_results.append((items[display[0]], items[display[1]], 
                                         str(temp)))
                    
    print('There are {total} BioThings objects returned for {ht}:'.format(\
                total = len(concise_results), ht = query))
    for display_info in concise_results:
        print('{0}, {1}, num of IDs: {2}'.format(display_info[0], display_info[1], display_info[2]))

In [3]:
def filter_table(df):
    """
    use _source and _method columns to remove rows (paths) from the dataframe
    works with Explain and Predict queries
    :param: pandas dataframe containing results from BTE FindConnection module, in table form
    
    Returns: filtered dataframe
    """
    ## key is the string to match to column, value is a list of strings to match to column values
    filter_out = {'_source': ['SEMMED', 'CTD', 'ctd', 'omia']   
#                   '_method': []  ## currently no method stuff I want to filter out
                 }
    ## SEMMED: text mining results wrong for PhenotypicFeature -> Gene
    ## CTD/ctd: results odd for MSUD -> ChemicalSubstance
    ## omia: results wrong or discontinued gene IDs for PhenotypicFeature -> Gene
    
    
    df_temp = df.copy()  ## so the original df isn't modified in-place
    for key,val in filter_out.items():
        ## find columns that match the key string
        columns = [i for i in df_temp.columns if key in i]
        ## iterate through each column
        for col in columns:
            ## iterate through each value to take out, check if string CONTAINS match. 
            ## only keep rows that don't contain the value
            for i in val:
                df_temp = df_temp[~ df_temp[col].str.contains(i, na = False)]

    return df_temp

In [4]:
def scoring_output(df, q_type):
    """
    score results based on whether query was Predict or Explain type, number of 
        intermediate nodes 
    :param: pandas dataframe containing results from BTE FindConnection module
    :param: string describing type of query (Predict or Explain)
    
    May flatten some edges, because score only counts one edge per 
        unique predicate / API / method (ignoring source and pubmed col)
    
    Predict queries: score each output node by counting # of paths
        from input nodes to it. Normalize by dividing by maximum
        possible # of paths
    Explain two-hop (one intermediate) queries: score each intermediate node by 
        counting # of paths (between input and output nodes) that include it. 
        Normalize by dividing by maximum possible # of paths    

    Explain one-hop (direct) queries: no need to score, prints message
    Other Explain queries (many-hops): currently not able to score, prints message     
    
    Returns: pandas series with scores, index is output_name
             or None (one-hop or many-hop Explain query)
    """
    df_temp = df.copy()  ## so no chance to mutate this   
    flag_direct = False  ## one-hop query or not
    ## use df_col to look quicker into columns
    df_col = set(df_temp.columns)
    
    ## ignore source and pubmed col in looking at unique edges 
    columns_drop = [col for col in df_col if (('_source' in col) or ('_pubmed' in col))]
    df_temp.drop(columns = columns_drop, inplace = True)    
    df_temp.drop_duplicates(inplace = True)
    
    ## check if query is one-hop or not
    if "node1_name" not in df_col:    ## name for first intermediate node layer
        flag_direct = True  
    
    if q_type == 'Explain':
        if flag_direct:   # one hop / no intermediates
            print('No valid node scoring for one-hop (direct) Explain queries.')
            return None
        ## if there are many-hops/intermediate layers
        elif "node2_name" in df_col:  ## name for 2nd intermed. node layer
            print('Cannot currently score many-hop Explain queries.')
            return None
        else:   ## two-hop / 1 intermediate layer
            ## count multi-edges to results (the intermediate node1 col)
            scores = df_temp.node1_name.value_counts() 
            ## to find the maximum-possible number of edges, look at non-result cols
            columns_drop = [col for col in df_col if 'node1' in col]
            df_temp.drop(columns = columns_drop, inplace = True)
            ## now look at number of unique combos for input, edge info, output
            df_temp.drop_duplicates(inplace = True)
            max_paths = df_temp.shape[0]            
            ## normalize scores by dividing each by max number of paths
            scores = scores / max_paths

    else:  ## Predict type query
        ## count multi-edges to results (the output col)
        scores = df_temp.output_name.value_counts()
        ## to find the maximum number of multi-edges, look at non-output col
        columns_drop = [col for col in df_temp.columns if 'output' in col]
        df_temp.drop(columns = columns_drop, inplace = True)
        ## now look at number of unique paths possible
        df_temp.drop_duplicates(inplace = True)
        max_paths = df_temp.shape[0]
        ## normalize scores by dividing each by max number of paths
        scores = scores / max_paths
            
    ## return scores as pandas dataframe, with rank
    scores = scores.to_frame(name = 'score') 
    scores['rank'] = scores['score'].rank(method = 'dense', ascending = False)
    return scores

In [5]:
## record when cell blocks are executed
print('The time that this notebook was executed is...')
print('Local time (PST, West Coast USA): ')
print(datetime.now())
print('UTC time: ')
print(datetime.utcnow())

The time that this notebook was executed is...
Local time (PST, West Coast USA): 
2020-11-26 17:57:39.141272
UTC time: 
2020-11-27 01:57:39.141408


## Specific use case #1: lung adenocarcinoma and brigatinib

This use case illustrates situation (1) discussed above in "Understanding the Question". We will use **lung adenocarcinoma** as our specific disease of interest. We will use **brigatinib** as our drug of interest. [Brigatinib](https://www.drugs.com/ppa/brigatinib.html) is an inhibitor of ALK, ROS1, IGF-1R, FLT3, and mutant-EGFR. This drug is indicated (currently approved) for use in ALK-positive metastatic non-small cell lung cancer (which includes lung adenocarcinoma as a subclass).

This drug has been [studied for its potential to help overcome acquired resistance to other EGFR-tyrosine kinase inhibitors (TKIs), which occurs when the cancer cells acquire specific EGFR mutations](https://pubmed.ncbi.nlm.nih.gov/32353596/). Brigatinib is [known to preferentially bind to triple-mutant-EGFR over wild-type EGFR, inhibiting it, and to inhibit growth (a synergistic effect) in *in vivo* studies when used with another anti-EGFR antibody treatment](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5355811/). 

We tackle the search for genes associated with brigatinib and lung adenocarcinoma using the query:  
* `Disease`lung adenocarcinoma  &rarr; results:`Gene` &larr; `ChemicalSubstance` brigatinib. 
* The query will return a graph object with entities as nodes and relationships as edges. We then use edge provenance information to **filter** the results. For each intermediate Disease node, we use the number of unique paths from the input nodes to that node to **score** it. The scores can then be used to sort the results.  

### Step 1: Find representation of "lung adenocarcinoma" in BTE

In this step, BioThings Explorer translates our query string "lung adenocarcinoma" into BioThings objects, which contain mappings to many common identifiers. We then pick the BioThings object that best matches what we want. 

Generally, the top result returned by the Hint module for your BioThings type of interest will match what you want, but you should confirm that using the identifiers shown. 


> BioThings types correspond to children and descendants of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `Disease` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle"). **However, [only a subset of the Biolink BiologicalEntity children / descendants are currently implemented in BTE](https://smart-api.info/portal/translator/metakg)**. More biomedical object types will be available as more knowledge sources (APIs) are added to the system. **Note that the type `BiologicalEntity` means any BioThings type currently implemented in BTE will be accepted.**

In [6]:
ht = Hint()  ## neater way to call this BTE module

## the human user gives this input
disease_starting_str = "lung adenocarcinoma"

disease_hint = ht.query(disease_starting_str)
hint_display(disease_starting_str, disease_hint)

There are 9 BioThings objects returned for lung adenocarcinoma:
Gene, lung adenocarcinoma-associated transcript 1, num of IDs: 2
Gene, lung adenocarcinoma associated transcript 1, num of IDs: 5
Gene, metastasis associated lung adenocarcinoma transcript 1, num of IDs: 5
Disease, lung occult adenocarcinoma, num of IDs: 3
Disease, lung adenocarcinoma, num of IDs: 4
Disease, papillary lung adenocarcinoma, num of IDs: 3
Disease, acinar lung adenocarcinoma, num of IDs: 3
Disease, lung colloid adenocarcinoma, num of IDs: 3
PhenotypicFeature, Lung adenocarcinoma, num of IDs: 4


Based on the information above, we'll pick the second `Disease` choice (indexed at 1) for our query because we want lung adenocarcinoma. We can look at identifier mappings inside this BioThings object. 

In [7]:
## the human user makes this choice, gives this input
disease_choice_type = 'Disease'
disease_choice_idx = 1

disease_hint_obj = disease_hint[disease_choice_type][disease_choice_idx]  
disease_hint_obj

{'MONDO': 'MONDO:0005061',
 'DOID': 'DOID:3910',
 'UMLS': 'C0152013',
 'name': 'lung adenocarcinoma',
 'MESH': 'C538231',
 'primary': {'identifier': 'MONDO',
  'cls': 'Disease',
  'value': 'MONDO:0005061'},
 'display': 'MONDO(MONDO:0005061) DOID(DOID:3910) UMLS(C0152013) MESH(C538231) name(lung adenocarcinoma)',
 'type': 'Disease'}

### Step 2: Find representation of "brigatinib" in BTE

In this step, BioThings Explorer translates our query string "brigatinib"  into BioThings objects, which contain mappings to many common identifiers. We then pick the BioThings object that best matches what we want. 

In [8]:
## the human user gives this input
drug_starting_str = "brigatinib"

drug_hint = ht.query(drug_starting_str)
hint_display(drug_starting_str, drug_hint)

There are 2 BioThings objects returned for brigatinib:
ChemicalSubstance, BRIGATINIB, num of IDs: 11
ChemicalSubstance, brigatinib, num of IDs: 2


All of these `ChemicalSubstance` entries seem to be the right object. We'll pick the entry with the most identifiers (indexed at 0) for our query. We can look at identifier mappings inside this BioThings object. 

In [9]:
## the human user makes this choice, gives this input
drug_choice_type = 'ChemicalSubstance'
drug_choice_idx = 0

drug_hint_obj = drug_hint[drug_choice_type][drug_choice_idx]  
drug_hint_obj

{'CHEMBL.COMPOUND': 'CHEMBL3545311',
 'DRUGBANK': 'DB12267',
 'PUBCHEM': 68165256,
 'UMLS': 'C3274459',
 'MESH': 'C000598580',
 'UNII': 'HYW8DB273J',
 'INCHIKEY': 'AILRADAXUVEEIR-UHFFFAOYSA-N',
 'INCHI': 'InChI=1S/C29H39ClN7O2P/c1-35-15-17-37(18-16-35)21-11-13-36(14-12-21)22-9-10-24(26(19-22)39-2)33-29-31-20-23(30)28(34-29)32-25-7-5-6-8-27(25)40(3,4)38/h5-10,19-21H,11-18H2,1-4H3,(H2,31,32,33,34)',
 'name': 'BRIGATINIB',
 'CAS': '1197953-54-0',
 'IUPAC': '[5-chloro-4-(2-dimethylphosphorylanilino)pyrimidin-2-yl]-[2-methoxy-4-[4-(4-methylpiperazino)piperidino]phenyl]amine',
 'formula': 'C29H39ClN7O2P',
 'primary': {'identifier': 'CHEMBL.COMPOUND',
  'cls': 'ChemicalSubstance',
  'value': 'CHEMBL3545311'},
 'display': 'CHEMBL.COMPOUND(CHEMBL3545311) DRUGBANK(DB12267) PUBCHEM(68165256) MESH(C000598580) UNII(HYW8DB273J) UMLS(C3274459) name(BRIGATINIB) CAS(1197953-54-0) IUPAC([5-chloro-4-(2-dimethylphosphorylanilino)pyrimidin-2-yl]-[2-methoxy-4-[4-(4-methylpiperazino)piperidino]phenyl]amine) 

### Step 3: lung adenocarcinoma &rarr; Gene &larr; brigatinib

In this section, we dynamically generate a knowledge graph with paths connecting lung adenocarcinoma and the drug brigatinib to `Genes` . We will then look at the `Genes` that have relationships with both lung adenocarcinoma and brigatinib. 

BTE performs the **query path planning** and **query path execution** by deconstructing the query into individual API calls, executing those API calls, and then assembling the results.

The code block below takes ~1 min 40 seconds to run. 

In [11]:
## the human user gives this input
q1_intermediate = 'Gene'

q1 = FindConnection(input_obj = disease_hint_obj,\
                     output_obj = drug_hint_obj, \
                    intermediate_nodes = q1_intermediate)
q1.connect(verbose = True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'lung adenocarcinoma' and 'BRIGATINIB'. Paths will have 1 intermediate node.

Intermediate node #1 will have these type constraints: Gene


========== QUERY #1 -- fetch all Gene entities linked to 'lung adenocarcinoma' ==========

==== Step #1: Query path planning ====

Because lung adenocarcinoma is of type 'Disease', BTE will query our meta-KG for APIs that can take 'Disease' as input and 'Gene' as output

BTE found 10 apis:

API 1. mgi_gene2phenotype(1 API call)
API 2. scibite(1 API call)
API 3. pharos(1 API call)
API 4. DISEASES(1 API call)
API 5. semmed_disease(15 API calls)
API 6. biolink(1 API call)
API 7. mydisease(1 API call)
API 8. hetio(1 API call)
API 9. scigraph(1 API call)
API 10. cord_disease(1 API call)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 1.1: https://pending.biothings.io/mgigene2phenot

In [12]:
# q1_r_graph = q1.fc.G  ## for changing the graph object to reflect the table
q1_r_paths_table = q1.display_table_view()

q1_type = re.findall("dispatcher.([a-zA-Z]+)'", str(type(q1.fc)))
q1_type = "".join(q1_type)  ## convert to string

# q1 = None  ## clear memory

We can see the number of Genes that were linked to both lung adenocarcinoma and to brigatinib and the total number of paths from the lung adenocarcinoma node to the brigatinib node.

In [13]:
## show number of unique intermediate nodes
print("There are {0} unique {1}s linked to both {2} and {3}.".format( \
    q1_r_paths_table.node1_name.nunique(), q1_intermediate, disease_starting_str, drug_starting_str))

## show number of paths from disease to drug
print("There are {0} unique paths between {1} and {2}.".format( \
    q1_r_paths_table.shape[0], disease_starting_str, drug_starting_str))

There are 5 unique Genes linked to both lung adenocarcinoma and brigatinib.
There are 25 unique paths between lung adenocarcinoma and brigatinib.


#### Filtering and scoring

Filtering involves using edge provenance, like the source this relationship came from and the method used to make this association, to filter out edges (removing nodes in the process). 

In [14]:
q1_r_paths_table = filter_table(q1_r_paths_table)

After filtering, there are fewer paths in the answer knowledge graph. 

In [15]:
## show number of unique intermediate nodes
print("There are {0} unique {1}s linked to both {2} and {3}.".format( \
    q1_r_paths_table.node1_name.nunique(), q1_intermediate, disease_starting_str, drug_starting_str))

## show number of paths from IBD to Imuran
print("There are {0} unique paths between {1} and {2}.".format( \
    q1_r_paths_table.shape[0], disease_starting_str, drug_starting_str))

There are 5 unique Genes linked to both lung adenocarcinoma and brigatinib.
There are 19 unique paths between lung adenocarcinoma and brigatinib.


The scoring process for two-hop Explain queries (the type of query we're using now, has one intermediate step): 

1. To score individual intermediate nodes (Genes), we first take a copy of the knowledge graph (KG) and remove some multi-edges. 
    * Each edge has predicate, API, method, source, and pubmed information. For scoring purposes, we will ignore pubmed and source information because APIs handle this information differently (returning multiple edges or single edges). 
2. We then count the number of edges to each intermediate node (from lung adenocarcinoma and brigatinib nodes).        
3. Finally, we "normalize" the score by dividing those counts by maximum-possible number of edges to an intermediate node (from lung adenocarcinoma and brigatinib nodes).

We can then see the top-scored intermediate nodes. A score of closer to 1 means that the many relationships link this node to lung adenocarcinoma and brigatinib. A score closer to 0 means that only a few relationships link this node to lung adenocarcinoma and brigatinib. 

In [16]:
## create scoring table for results
q1_scoring = scoring_output(q1_r_paths_table, q1_type)

q1_scoring

,score,rank
EGFR,1.00,1.0
ROS1,0.75,2.0
ALK,0.75,2.0
MET,0.25,3.0
ERBB2,0.25,3.0


Different knowledge sources (APIs) were called in different parts of the query. 

In the first part of the query (lung adenocarcinoma &rarr; Gene), the following APIs returned results and the following predicates (semantic relationships) were found.

In [17]:
## show that the APIs use different predicates
q1_r_paths_table[['pred1_api', 'pred1']].drop_duplicates().sort_values(by = ['pred1_api', 'pred1'])

,pred1_api,pred1
2,Automat CORD19 Scibite API,related_to
9,Automat CORD19 Scigraph API,related_to
1,Automat PHAROS API,related_to
0,CORD Disease API,related_to


In the second part of the query (brigatinib &rarr; Gene), the following APIs returned results and the following predicates (semantic relationships) were found.

In [18]:
## show that the APIs use different predicates
q1_r_paths_table[['pred2_api', 'pred2']].drop_duplicates().sort_values(by = ['pred2_api', 'pred2'])

,pred2_api,pred2
0,MyChem.info API,physically_interacts_with


[TO DO: Actual code depends on graph object (ReasonerStd object or Networkx MultiDiGraph) used.]

We then add the scores and score provenance information to the graph object that will be returned to the ARS. These stored as node attributes on the Gene nodes of the "answer graph". 

In [18]:
# nx.set_node_attributes(q1_r_graph, values = q1_scoring.to_dict(), \
#                        name = 'score')
# nx.set_node_attributes(q1_r_graph, values = '(0-1]', name = 'score_range')
# nx.set_node_attributes(q1_r_graph, values = 'normalized path count', name = 'score_method')
# ## what is a good score, larger or smaller? 
# nx.set_node_attributes(q1_r_graph, values = 'larger', name = 'score_better_direction')  

# ## example of Gene node object with score-related attributes
# q1_r_graph.nodes()['NDUFS1']

### Evaluate results 

Lung adenocarcinoma is linked to [mutations or fusions in all of the gene](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5542927/) found by BTE. 

As mentioned above, [brigatinib](https://www.drugs.com/ppa/brigatinib.html) is an inhibitor of ALK, ROS1, and mutant-EGFR and is indicated (currently approved) for use in ALK-positive metastatic non-small cell lung cancer (which includes lung adenocarcinoma as a subclass). 

This drug has also showed [*in vitro* and clinical activity against cancer cells expressing EML4-ALK mutant forms associated with ALK inhibitor resistance](https://www.drugs.com/ppa/brigatinib.html). 

A quick literature search did not find links between brigatinib and MET. 

In [19]:
q1_scoring

,score,rank
EGFR,1.00,1.0
ROS1,0.75,2.0
ALK,0.75,2.0
MET,0.25,3.0
ERBB2,0.25,3.0


## Specific use case #2: inflammatory bowel disease and imuran

This use case illustrates situation (3) discussed above in "Understanding the Question". We will use **inflammatory bowel disease (IBD)** as our specific disease of interest. We will use **Imuran** as our drug of interest. [Imuran / azathioprine](https://www.uptodate.com/contents/azathioprine-drug-information) has been [commonly used **off-label** to treat Crohn disease (a form of IBD)](https://www.mayoclinic.org/diseases-conditions/crohns-disease/diagnosis-treatment/drc-20353309). 

However, people taking this drug [should be tested for](https://www.drugs.com/pro/imuran.html)...
* TPMT deficiency: ~10% of patients of European/African ancestry have one loss-of-function allele (intermediate metabolizers)
* NUDT15 deficiency: 2% of patients of East Asian ancestry have two loss-of-function alleles in this gene, and ~21% have one loss-of-function allele  

We tackle the search for genes associated with inflammatory bowel disease and Imuran using the query:  
* `Disease`inflammatory bowel disease &rarr; results:`Gene` &larr; `ChemicalSubstance` imuran. 
* The query will return a graph object with entities as nodes and relationships as edges. We then use edge provenance information to **filter** the results. For each intermediate Disease node, we use the number of unique paths from the input nodes to that node to **score** it. The scores can then be used to sort the results.  

### Step 1: Find representation of "inflammatory bowel disease" in BTE

In this step, BioThings Explorer translates our query string "IBD" into BioThings objects, which contain mappings to many common identifiers. We then pick the BioThings object that best matches what we want. 

Generally, the top result returned by the Hint module for your BioThings type of interest will match what you want, but you should confirm that using the identifiers shown. 


> BioThings types correspond to children and descendants of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `Disease` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle"). **However, [only a subset of the Biolink BiologicalEntity children / descendants are currently implemented in BTE](https://smart-api.info/portal/translator/metakg)**. More biomedical object types will be available as more knowledge sources (APIs) are added to the system. **Note that the type `BiologicalEntity` means any BioThings type currently implemented in BTE will be accepted.**

In [20]:
ht = Hint()  ## neater way to call this BTE module

## the human user gives this input
disease2_starting_str = "IBD"

disease2_hint = ht.query(disease2_starting_str)
hint_display(disease2_starting_str, disease2_hint)

There are 12 BioThings objects returned for IBD:
ChemicalSubstance, IBD-78, num of IDs: 2
ChemicalSubstance, IBD-78, (Z)-, num of IDs: 6
ChemicalSubstance, IBD-78 HYDROCHLORIDE, num of IDs: 2
ChemicalSubstance, IBD-78 ACETATE, num of IDs: 2
ChemicalSubstance, IBD-78, (E)-, num of IDs: 6
Disease, irritable bowel syndrome, num of IDs: 5
Disease, inflammatory bowel disease, num of IDs: 4
Disease, IL21-related infantile inflammatory bowel disease, num of IDs: 4
Disease, immune dysregulation-inflammatory bowel disease-arthritis-recurrent infections syndrome, num of IDs: 3
Disease, multiple intestinal atresia, num of IDs: 6
Pathway, Inflammatory bowel disease (IBD) - Homo sapiens (human), num of IDs: 1
Pathway, Inflammatory bowel disease (IBD) - Mus musculus (mouse), num of IDs: 1


Based on the information above, we'll pick the second `Disease` choice (indexed at 1) for our query. We can look at identifier mappings inside this BioThings object. 

In [21]:
## the human user makes this choice, gives this input
disease2_choice_type = 'Disease'
disease2_choice_idx = 1

disease2_hint_obj = disease2_hint[disease2_choice_type][disease2_choice_idx]  
disease2_hint_obj

{'MONDO': 'MONDO:0005265',
 'DOID': 'DOID:0050589',
 'UMLS': 'C0021390',
 'name': 'inflammatory bowel disease',
 'MESH': 'D015212',
 'primary': {'identifier': 'MONDO',
  'cls': 'Disease',
  'value': 'MONDO:0005265'},
 'display': 'MONDO(MONDO:0005265) DOID(DOID:0050589) UMLS(C0021390) MESH(D015212) name(inflammatory bowel disease)',
 'type': 'Disease'}

### Step 2: Find representation of "Imuran" in BTE

In this step, BioThings Explorer translates our query string "imuran"  into BioThings objects, which contain mappings to many common identifiers. We then pick the BioThings object that best matches what we want. 

In [22]:
## the human user gives this input
drug2_starting_str = "imuran"

drug2_hint = ht.query(drug2_starting_str)
hint_display(drug2_starting_str, drug2_hint)

There are 3 BioThings objects returned for imuran:
ChemicalSubstance, AZATHIOPRINE, num of IDs: 13
ChemicalSubstance, Imuran, num of IDs: 2
ChemicalSubstance, AZATHIOPRINE SODIUM, num of IDs: 8


2020-11-26 update: Hint module results changed so the pick selection is changed. All of these `ChemicalSubstance` entries seem to be the right object. We'll pick the entry with the most identifiers (first option, indexed at 0) for our query. We can look at identifier mappings inside this BioThings object. 

In [23]:
## the human user makes this choice, gives this input
drug2_choice_type = 'ChemicalSubstance'
drug2_choice_idx = 0

drug2_hint_obj = drug2_hint[drug2_choice_type][drug2_choice_idx]  
drug2_hint_obj

{'CHEMBL.COMPOUND': 'CHEMBL1542',
 'DRUGBANK': 'DB00993',
 'PUBCHEM': 2265,
 'CHEBI': 'CHEBI:2948',
 'UMLS': 'C0004482',
 'MESH': 'D001379',
 'UNII': 'AM94R510MS',
 'INCHIKEY': 'LMEKQMALGUDUQG-UHFFFAOYSA-N',
 'INCHI': 'InChI=1S/C9H7N7O2S/c1-15-4-14-7(16(17)18)9(15)19-8-5-6(11-2-10-5)12-3-13-8/h2-4H,1H3,(H,10,11,12,13)',
 'KEGG': 'C06837',
 'name': 'AZATHIOPRINE',
 'CAS': '446-86-6',
 'IUPAC': '6-[(3-methyl-5-nitro-imidazol-4-yl)thio]-7H-purine',
 'formula': 'C9H7N7O2S',
 'primary': {'identifier': 'CHEBI',
  'cls': 'ChemicalSubstance',
  'value': 'CHEBI:2948'},
 'display': 'CHEBI(CHEBI:2948) CHEMBL.COMPOUND(CHEMBL1542) DRUGBANK(DB00993) PUBCHEM(2265) MESH(D001379) UNII(AM94R510MS) UMLS(C0004482) name(AZATHIOPRINE) CAS(446-86-6) IUPAC(6-[(3-methyl-5-nitro-imidazol-4-yl)thio]-7H-purine) formula(C9H7N7O2S)',
 'type': 'ChemicalSubstance'}

### Step 3: inflammatory bowel disease &rarr; Gene &larr; imuran

In this section, we dynamically generate a knowledge graph with paths connecting inflammatory bowel disease and the drug imuran to `Genes` . We will then look at the `Genes` that have relationships with both inflammatory bowel disease and imuran. 

BTE performs the **query path planning** and **query path execution** by deconstructing the query into individual API calls, executing those API calls, and then assembling the results.

The code block below takes ~1 min 50 seconds to run. 

In [24]:
## the human user gives this input
q2_intermediate = 'Gene'

q2 = FindConnection(input_obj = disease2_hint_obj,\
                     output_obj = drug2_hint_obj, \
                    intermediate_nodes = q2_intermediate)
q2.connect(verbose = True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'inflammatory bowel disease' and 'AZATHIOPRINE'. Paths will have 1 intermediate node.

Intermediate node #1 will have these type constraints: Gene


========== QUERY #1 -- fetch all Gene entities linked to 'inflammatory bowel disease' ==========

==== Step #1: Query path planning ====

Because inflammatory bowel disease is of type 'Disease', BTE will query our meta-KG for APIs that can take 'Disease' as input and 'Gene' as output

BTE found 10 apis:

API 1. mgi_gene2phenotype(1 API call)
API 2. scibite(1 API call)
API 3. pharos(1 API call)
API 4. DISEASES(1 API call)
API 5. semmed_disease(15 API calls)
API 6. biolink(1 API call)
API 7. mydisease(1 API call)
API 8. hetio(1 API call)
API 9. scigraph(1 API call)
API 10. cord_disease(1 API call)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 6.1: https://mydisease.in

In [25]:
# q2_r_graph = q2.fc.G  ## for changing the graph object to reflect the table
q2_r_paths_table = q2.display_table_view()

q2_type = re.findall("dispatcher.([a-zA-Z]+)'", str(type(q2.fc)))
q2_type = "".join(q2_type)  ## convert to string

# q1 = None  ## clear memory

We can see the number of Genes that were linked to both lung adenocarcinoma and to brigatinib and the total number of paths from the lung adenocarcinoma node to the brigatinib node.

In [26]:
## show number of unique intermediate nodes
print("There are {0} unique {1}s linked to both {2} and {3}.".format( \
    q2_r_paths_table.node1_name.nunique(), q2_intermediate, disease2_starting_str, drug2_starting_str))

## show number of paths from disease to drug
print("There are {0} unique paths between {1} and {2}.".format( \
    q2_r_paths_table.shape[0], disease2_starting_str, drug2_starting_str))

There are 71 unique Genes linked to both IBD and imuran.
There are 324 unique paths between IBD and imuran.


#### Filtering and scoring

Filtering involves using edge provenance, like the source this relationship came from and the method used to make this association, to filter out edges (removing nodes in the process). 

In [27]:
q2_r_paths_table = filter_table(q2_r_paths_table)

After filtering, there are fewer paths in the answer knowledge graph. 

In [28]:
## show number of unique intermediate nodes
print("There are {0} unique {1}s linked to both {2} and {3}.".format( \
    q2_r_paths_table.node1_name.nunique(), q2_intermediate, disease2_starting_str, drug2_starting_str))

## show number of paths from disease to drug
print("There are {0} unique paths between {1} and {2}.".format( \
    q2_r_paths_table.shape[0], disease2_starting_str, drug2_starting_str))

There are 27 unique Genes linked to both IBD and imuran.
There are 67 unique paths between IBD and imuran.


The scoring process for two-hop Explain queries (the type of query we're using now, has one intermediate step): 

1. To score individual intermediate nodes (Genes), we first take a copy of the knowledge graph (KG) and remove some multi-edges. 
    * Each edge has predicate, API, method, source, and pubmed information. For scoring purposes, we will ignore pubmed and source information because APIs handle this information differently (returning multiple edges or single edges). 
2. We then count the number of edges to each intermediate node (from lung adenocarcinoma and brigatinib nodes).        
3. Finally, we "normalize" the score by dividing those counts by maximum-possible number of edges to an intermediate node (from lung adenocarcinoma and brigatinib nodes).

We can then see the top-scored intermediate nodes. A score of closer to 1 means that the many relationships link this node to lung adenocarcinoma and brigatinib. A score closer to 0 means that only a few relationships link this node to lung adenocarcinoma and brigatinib. 

In [29]:
## create scoring table for results
q2_scoring = scoring_output(q2_r_paths_table, q2_type)

q2_scoring.head(10)

,score,rank
TPMT,0.521739,1.0
TNF,0.260870,2.0
CRP,0.173913,3.0
NUDT15,0.173913,3.0
FOXP3,0.173913,3.0
CAT,0.130435,4.0
CD4,0.130435,4.0
ITPA,0.130435,4.0
AHR,0.130435,4.0
CAST,0.086957,5.0


Different knowledge sources (APIs) were called in different parts of the query. 

In the first part of the query (lung adenocarcinoma &rarr; Gene), the following APIs returned results and the following predicates (semantic relationships) were found.

In [30]:
## show that the APIs use different predicates
q1_r_paths_table[['pred1_api', 'pred1']].drop_duplicates().sort_values(by = ['pred1_api', 'pred1'])

,pred1_api,pred1
2,Automat CORD19 Scibite API,related_to
9,Automat CORD19 Scigraph API,related_to
1,Automat PHAROS API,related_to
0,CORD Disease API,related_to


In the second part of the query (brigatinib &rarr; Gene), the following APIs returned results and the following predicates (semantic relationships) were found.

In [31]:
## show that the APIs use different predicates
q1_r_paths_table[['pred2_api', 'pred2']].drop_duplicates().sort_values(by = ['pred2_api', 'pred2'])

,pred2_api,pred2
0,MyChem.info API,physically_interacts_with


[TO DO: Actual code depends on graph object (ReasonerStd object or Networkx MultiDiGraph) used.]

We then add the scores and score provenance information to the graph object that will be returned to the ARS. These stored as node attributes on the Gene nodes of the "answer graph". 

In [32]:
# nx.set_node_attributes(q1_r_graph, values = q1_scoring.to_dict(), \
#                        name = 'score')
# nx.set_node_attributes(q1_r_graph, values = '(0-1]', name = 'score_range')
# nx.set_node_attributes(q1_r_graph, values = 'normalized path count', name = 'score_method')
# ## what is a good score, larger or smaller? 
# nx.set_node_attributes(q1_r_graph, values = 'larger', name = 'score_better_direction')  

# ## example of Gene node object with score-related attributes
# q1_r_graph.nodes()['NDUFS1']

### Evaluate results 

As mentioned above, the known answers are TPMT and NUDT15. **These two genes are highly-scored results of the query `Disease` inflammatory bowel disease &rarr; `Gene` &larr; `ChemicalSubstance` Imuran**. The information supporting this answer came from multiple APIs. 

In [32]:
known_answers = ['TPMT', 'NUDT15']

In [33]:
print('Scoring from the IBD -> Gene <- Imuran query')

## reset index to show placement of genes
q2_scoring_df = q2_scoring.reset_index().rename(columns = {'index': 'node1_name'})
q2_scoring_df[q2_scoring_df.node1_name.isin(known_answers)]

## show APIs/sources for the underlying info for these answers
q2_r_paths_table[q2_r_paths_table.node1_name.isin(known_answers)][['pred1_api']].drop_duplicates()
q2_r_paths_table[q2_r_paths_table.node1_name.isin(known_answers)][['pred2_api']].drop_duplicates()

Scoring from the IBD -> Gene <- Imuran query


,node1_name,score,rank
0,TPMT,0.521739,1.0
3,NUDT15,0.173913,3.0


,pred1_api
100,DISEASES API
101,mydisease.info API
103,Automat PHAROS API
105,BioLink API


,pred2_api
100,MyChem.info API
151,Automat CORD19 Scibite API
158,Automat HMDB API
